In [129]:
import torch
import PyPDF2
import nltk
from scipy.spatial.distance import cosine
import numpy as np

# Extracting PDF

In [40]:
file = r"Reports\EliLilly_2021_Environmental_ESG_report.pdf"

In [41]:
pdfReader = PyPDF2.PdfFileReader(file) 

In [42]:
n = len(pdfReader.pages)
txt = ''
for i in range(n):
    temp = pdfReader.pages[i].extractText()
    txt = txt + temp

In [43]:
tokens = nltk.sent_tokenize(txt.replace('\n', ''))

In [44]:
#checking extracted sentences
for t in tokens:
    print(t)

7/7/22, 10:29 AM Environmental | 2021 ESG Report | Eli Lilly and Companyhttps://esg.lilly.com/environmental#tab-control-tab4 1/7Our purpose, to make life better, includes protecting and preserving the world we live in.
Making medicines requires the use of valuableresources including energy, water and raw materials.
We’re committed to reducing our environmental footprint across the life cycles ofour products and our supply chain.
T o track our progress, we measure and manage energy and water use, greenhouse gas (GHG)emissions and the generation of waste and wastewater thr oughout our manufacturing pr ocess.
Lilly manages health, saf ety and theenvironment (HSE) under a uni ed governance structure.Our 2030 Envir onmental GoalsClimateCarbon neutralin our own operations100%renewable electricityEnhancefull value-chain emissions reportingWasteZerowaste to land ll100%of plastic waste repurposed forbene cial use with at least 90%recycled or reusedIntegrate sustainabilityinto product and packag

# Preprocessing String

In [45]:
#giving new name for preprocess strings
#keeping only sentences with length greater than 3
pre_tokens = []
for sen in tokens:
    if len(sen.split(' ')) > 3:
        pre_tokens.append(sen)
   

# Importing Climate Bert

In [46]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

model = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-f",output_hidden_states = True)

torch.Size([768])

In [20]:
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50500, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

# Sanity Checks

In [ ]:
#get vocab of Bert Model



#get consine distance from vocab to keywords




# Getting the input embeddings of the tokens

In [73]:
# input embedding matrix
a = model.roberta.get_input_embeddings().weight

# Finding the top 20 tokens near keywords from each token 

In [85]:
keywords = ['Electric vehicle',
'Solar',
'Wind',
'Hydroelectric' ,
'Nuclear',
'REC',
'Efficiency',
'Deforestation',
'Afforestation',
'carbon',
'credit',
'capture',
'sequestration',
'storage',
'Hydrogen',
'Geothermal',
'Biomass',
'Renewable', 
'Energy',
'emissions',
'reforestation',
'Decreased', 
'Reduced']

In [1]:
target = []
for ids,sen in enumerate(keywords):
    
    marked_text = "[CLS] " + sen + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)


    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


    #creating segment ids for the sentence
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    hidden_states = []
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

        hidden_states = outputs[-1]


    #print('Tensor shape for each layer: ', hidden_states[0].size())

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # token_embeddings.size()

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(sum_vec)
        
    target.append(torch.stack(token_vecs_cat, dim = 0).mean(dim = 0))

NameError: name 'keywords' is not defined

In [168]:
len(token_vecs_cat)

9

In [155]:
target[0]

tensor([[-0.1040,  0.1478, -0.0783,  ..., -0.0496, -0.0153, -0.3452],
        [ 0.2337,  2.1944,  1.5224,  ...,  0.4638,  0.1454, -1.0760],
        [ 0.0764,  2.6973,  0.8157,  ..., -1.8009, -0.6251, -1.4696],
        ...,
        [ 0.3301,  0.0796,  0.9269,  ..., -1.4880,  0.2279, -1.4488],
        [-0.0771,  1.2568,  1.3948,  ..., -0.5472, -0.2325, -1.6407],
        [-0.6514, -2.0505,  1.3429,  ..., -2.2260, -1.0185, -1.0036]])

In [145]:
#We normalize embeddings, so that euclidian distance is equivalent to cosine distance
all_embeddings = a.detach().numpy()
normed_embeddings = (all_embeddings.T / (all_embeddings**2).sum(axis=1) ** 0.5).T

In [148]:
from sklearn.neigbhbors import KDTree
#making kd tree
indexer = KDTree(normed_embeddings)

In [149]:
#querying tree according to the kw vectors
indexer.query(target[0].reshape(1, -1),return_distance = False, k = 20)

array([[50264,     0,     2,     4,     5, 50118,    20,    11,   133,
            9,    16, 50109, 50091,     6,     7,    13, 50190, 50102,
        50061,    32]], dtype=int64)

In [150]:
result = []
vocab = np.array(list(tokenizer.vocab.keys()))
for emb in zip(target,keywords):
    top_20 = indexer.query(emb[0].reshape(1, -1),return_distance = False, k = 20)
    print(top_20)
    result.append([emb[1],vocab[top_20]])
    

[[50264     0     2     4     5 50118    20    11   133     9    16 50109
  50091     6     7    13 50190 50102 50061    32]]
[[50264     0     2     4     5    20 50118    11   133     9    16     6
  50109     7 50091    36    13 50190    32 50102]]
[[50264     0     2     4     5    20 50118     9   133    11    16 50109
      7     6    13 50091    36    32 50190 50061]]
[[50264     0     2     4    20     5 50118   133     9    11    16     6
  50091 50109     7    32 50190    36    34    13]]
[[50264     0     2     4     5 50118    20    11   133     9    16 50109
      6 50091     7    36    13 50102 50190    34]]
[[50264     0     2     4     5 50118    20    11     9    16   133 50109
      6 50091     7    32 50102    36    13 50241]]
[[50264     0     2     4     5    20 50118    11     9    16   133 50109
      6     7 50190 50091 50102    13 50061    32]]
[[50264     0     2     4     5    20    11     9 50118   133    16 50109
      7 50091     6    13 50190 50102    32 

In [151]:
result

[['Electric vehicle',
  array([['bite', 'ĠThai', 'Ġcrackdown', 'Esc', 'Ġ200', 'ĠGaddafi', 'ĠTHC',
          'Ġprett', 'ĠDry', 'Ġinf', 'Ġclues', 'ĠGrant', 'Ġevangel', 'hall',
          'Ġdental', 'enabled', 'Ġvolley', 'Ġfilings', 'Ġcrater',
          'Ġprotagonist']], dtype='<U128')],
 ['Solar',
  array([['bite', 'ĠThai', 'Ġcrackdown', 'Esc', 'Ġ200', 'ĠTHC', 'ĠGaddafi',
          'Ġprett', 'ĠDry', 'Ġinf', 'Ġclues', 'hall', 'ĠGrant', 'Ġdental',
          'Ġevangel', 'ĠHaving', 'enabled', 'Ġvolley', 'Ġprotagonist',
          'Ġfilings']], dtype='<U128')],
 ['Wind',
  array([['bite', 'ĠThai', 'Ġcrackdown', 'Esc', 'Ġ200', 'ĠTHC', 'ĠGaddafi',
          'Ġinf', 'ĠDry', 'Ġprett', 'Ġclues', 'ĠGrant', 'Ġdental', 'hall',
          'enabled', 'Ġevangel', 'ĠHaving', 'Ġprotagonist', 'Ġvolley',
          'Ġcrater']], dtype='<U128')],
 ['Hydroelectric',
  array([['bite', 'ĠThai', 'Ġcrackdown', 'Esc', 'ĠTHC', 'Ġ200', 'ĠGaddafi',
          'ĠDry', 'Ġinf', 'Ġprett', 'Ġclues', 'hall', 'Ġevangel', 'ĠGrant'

In [176]:
#Solar
solar_emb = all_embeddings[tokenizer.vocab['Solar']]
top_20 = indexer.query(solar_emb.reshape(1, -1),return_distance = False, k = 20)
vocab[top_20]

array([['Ġbrewery', 'ĠStatic', 'existence', 'Ġsublime', 'Ġsetup',
        'Ġearly', 'ĠAunt', 'ĠSk', 'ingu', 'SE', 'Double', 'Ġstrengths',
        'Ġirritating', 'oday', 'Ġcor', 'Ġtransmitting', 'Ġlied',
        'Ġgunshots', 'Ġgraphical', 'Ġtimet']], dtype='<U128')

In [177]:
#carbon
solar_emb = all_embeddings[tokenizer.vocab['carbon']]
top_20 = indexer.query(solar_emb.reshape(1, -1),return_distance = False, k = 20)
vocab[top_20]

array([['Ġtilted', 'Ġmicrophones', 'ĠPhillies', 'Ġrisking', 'LEY',
        'redo', 'Ġbanana', 'reenshot', 'Ġnecessarily', 'Ġcig', 'itable',
        'ĠHog', 'ĠSEC', 'Ġfortunate', 'Ġencoded', 'Ġ354', 'Ġprince',
        'knit', 'Ġinsulted', 'Ġshown']], dtype='<U128')

In [178]:
#Energy
solar_emb = all_embeddings[tokenizer.vocab['Energy']]
top_20 = indexer.query(solar_emb.reshape(1, -1),return_distance = False, k = 20)
vocab[top_20]

array([['Ġirritating', 'ĠMeridian', 'Ġ354', 'Ġprecept', 'ĠLOW',
        'reduced', 'Double', 'ĠIsles', 'Ġkb', 'Offline', 'Ġanimation',
        'ĠImprovement', 'ĠDiego', 'ackers', 'ĠFans', '2007', 'ingu',
        'Ġdiscard', 'ise', 'Ġbrewery']], dtype='<U128')

In [180]:
#Renewable
solar_emb = all_embeddings[tokenizer.vocab['forest']]
top_20 = indexer.query(solar_emb.reshape(1, -1),return_distance = False, k = 20)
vocab[top_20]

array([['Ġcontam', 'Ġcleaned', 'Ġmetabol', 'ĠBay', 'ĠMotion',
        'Ġeducation', 'ĠPhoto', 'Ġswords', 'andals', 'ign', 'ĠHispanics',
        'ĠPRESIDENT', 'Ġbothered', 'ĠPf', 'Ġevolution', 'ĠACT',
        'Ġcrates', 'ĠBened', 'Ġret', 'Ġlogistical']], dtype='<U128')

In [31]:
# for sen in tokens[0]:
from collections import defaultdict
chest = defaultdict()

for ids,sen in enumerate(pre_tokens):
    
    marked_text = "[CLS] " + sen + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)


    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


    #creating segment ids for the sentence
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    hidden_states = []
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

        hidden_states = outputs[-1]


    #print('Tensor shape for each layer: ', hidden_states[0].size())

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # token_embeddings.size()

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(cat_vec)

    # take mean of all tensors in token_vecs_cat
    chest[ids] = torch.stack(token_vecs_cat, dim = 0).mean(dim = 0)
    
    


In [8]:
sen = tokens[2]
marked_text = "[CLS] " + sen + " [SEP]"
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)


# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


#creating segment ids for the sentence
segments_ids = [1] * len(tokenized_text)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

hidden_states = []
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

    hidden_states = outputs[-1]


#print('Tensor shape for each layer: ', hidden_states[0].size())

token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

# token_embeddings.size()

# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

# take mean of all tensors in token_vecs_cat
# chest[ids] = torch.stack(token_vecs_cat, dim = 0).mean(dim = 0)

NameError: name 'tokenizer' is not defined

In [22]:
token_vecs_cat

[tensor([-0.0227,  0.0696, -0.0534,  ...,  0.0638,  0.0382, -0.0609]),
 tensor([-0.1132, -0.0195,  0.0453,  ...,  0.2379,  0.2747,  0.0322]),
 tensor([-0.0088,  0.0372, -0.0074,  ..., -0.3526,  0.0089, -0.2931]),
 tensor([-0.2523, -0.1380,  0.1747,  ..., -0.4824, -0.0483, -0.1880]),
 tensor([-0.0521, -0.2170,  0.1018,  ..., -0.0007, -0.3277, -0.2686]),
 tensor([ 0.0575,  0.0659, -0.0555,  ...,  0.3185, -0.1242,  0.2988]),
 tensor([-0.1485, -0.1143, -0.2501,  ...,  0.4499,  0.0978,  0.1220]),
 tensor([-0.0625,  0.0650, -0.0694,  ...,  0.1919,  0.1436, -0.0012]),
 tensor([-0.3260, -0.0296, -0.2715,  ...,  0.3067,  0.1489,  0.0349]),
 tensor([ 0.0193,  0.0814, -0.0041,  ..., -0.0198,  0.4140,  0.0816]),
 tensor([-0.1979,  0.0909, -0.2429,  ..., -0.1548, -0.6206, -0.3269]),
 tensor([-0.2348, -0.1096, -0.0956,  ..., -0.3618,  0.4425, -0.2723]),
 tensor([ 0.0234, -0.0267, -0.1142,  ..., -0.1734,  0.3193, -0.0743]),
 tensor([ 0.0307, -0.0044, -0.1531,  ..., -0.2912,  0.4400, -0.1886]),
 tenso

# Generating word embedding of the keywords

In [32]:
keywords = ['Electric vehicle',
'Solar',
'Wind',
'Hydroelectric' ,
'Nuclear',
'REC',
'Efficiency',
'Deforestation',
'Afforestation',
'carbon',
'credit',
'capture',
'sequestration',
'storage',
'Hydrogen',
'Geothermal',
'Biomass',
'Renewable', 
'Energy',
'emissions',
'reforestation',
'Decreased', 
'Reduced']
print(keywords)


['Electric vehicle', 'Solar', 'Wind', 'Hydroelectric', 'Nuclear', 'REC', 'Efficiency', 'Deforestation', 'Afforestation', 'carbon', 'credit', 'capture', 'sequestration', 'storage', 'Hydrogen', 'Geothermal', 'Biomass', 'Renewable', 'Energy', 'emissions', 'reforestation', 'Decreased', 'Reduced']


In [33]:
target = []

for ids,sen in enumerate(keywords):
    
    marked_text = "[CLS] " + sen + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)


    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)


    #creating segment ids for the sentence
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    hidden_states = []
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel

        hidden_states = outputs[-1]


    #print('Tensor shape for each layer: ', hidden_states[0].size())

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # token_embeddings.size()

    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(cat_vec)
        
    target.append(torch.stack(token_vecs_cat, dim = 0).mean(dim = 0))
    
    

In [34]:
#getting the embedding representation
grader = torch.stack(target, dim = 0).mean(dim = 0)

# Finding cosine distance 

In [35]:
tracker = []
for key,val in chest.items():
    tracker.append([key,1 - cosine(val, grader)])

In [36]:
tracker.sort(key = lambda val : val[1],reverse=True)

# Ranking sentences

In [38]:
for ele in tracker:
    print(pre_tokens[ele[0]])

Additional historical data areavailable in our reporting archives.
These improvements were made while production volume increased.
Sesto implemented new drive-belt technology on airhandlers to improve e ciency.
In 2021, we did not purchase any carbon offsets.
We have reduced GHG emissions at keyfacilities by leveraging solar energy.
Our site inKinsale, Ireland replaced  xed speed pump motors with variable speed motors.
In Indianapolis, a chiller was r eplacedwith a new, higher e ciency chiller.
For smaller locations not billed directly to Lilly, data are estimated based on square footage.3.
We haveintroduced hybrid or electric  eet v ehicles in several geographies where infrastructure is available.
These projects include chiller replacements,building air handler optimizations and air  ow r eduction initiatives.
Reducing our energy and emissions by making our overall operations more e cient 2.
In 2021, we achie ved a 9% absoluteemissions reduction versus 2020.
These data will also be us